In [21]:
pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install -U langchain-community langchain-huggingface chromadb sentence-transformers transformers accelerate protobuf==4.25.3

In [ ]:
!pip uninstall -y transformers
!pip uninstall -y transformers[sentencepiece]

In [ ]:
!pip install transformers==4.41.2
!pip install sentencepiece

In [ ]:
!pip install accelerate safetensors

In [26]:
from langchain.document_loaders import DirectoryLoader

In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline

In [28]:
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma

In [29]:
# =====================
# 1. Load and split data
# =====================
data_path = r"C:\Centraly_Project\Data\info.md"

loader = TextLoader(data_path, encoding="utf-8")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True
)
chunks = text_splitter.split_documents(documents)

print(f"Loaded {len(chunks)} chunks")


Loaded 83 chunks


In [30]:
# =====================
# 2. Create embeddings + ChromaDB
# =====================
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = Chroma.from_documents(chunks, embeddings, persist_directory="./chroma_db")
print("✅ Data stored in Chroma")

RuntimeError: Failed to import transformers.models.chameleon.configuration_chameleon because of the following error (look up to see its traceback):
No module named 'transformers.models.chameleon.configuration_chameleon'

In [ ]:
# =====================
# 3. Setup LLM (GPT-J from HuggingFace)
# =====================
llm = pipeline(
    "text-generation",
    model="EleutherAI/gpt-j-6B",
    device=-1,  # -1 = CPU, or 0 = GPU if available
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7
)

config.json:   0%|          | 0.00/930 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/24.2G [00:00<?, ?B/s]

In [1]:
# =====================
# 4. Ask a question
# =====================
query = "ما هي خدمات شركة الشفاء الرقمية؟"

# Step 1: Get relevant chunks
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
docs = retriever.get_relevant_documents(query)

context = "\n\n".join([doc.page_content for doc in docs])

# Step 2: Build prompt
prompt = f"السؤال: {query}\n\nالمعلومات المتاحة:\n{context}\n\nالإجابة:"

# Step 3: Get answer from LLM
response = llm(prompt)[0]["generated_text"]

print("\n--- Final Answer ---")
print(response)

NameError: name 'vectorstore' is not defined